In [1]:
#Read in Homicide csv

In [2]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import requests
import json

# Google developer API key
from config import gkey

# Access maps with unique API key
#gmaps.configure(api_key=gkey)

In [3]:
homicide_path= "Homicide.csv"

In [4]:
homicide_rawdata = pd.read_csv(homicide_path)

In [5]:
homicide_rawdata.head()

,X,Y,Index_,Event_Unique_ID,Occurrence_year,Division,Homicide_Type,Occurrence_Date,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.392853,43.685028,1,111878,2004,D53,Other,2004-01-03T05:00:00.000Z,98,Rosedale-Moore Park (98),43.685028,-79.392853,1
1,-79.234962,43.781536,2,125755,2004,D42,Shooting,2004-01-08T05:00:00.000Z,137,Woburn (137),43.781536,-79.234962,2
2,-79.206894,43.810860,3,136086,2004,D42,Shooting,2004-01-08T05:00:00.000Z,132,Malvern (132),43.810860,-79.206894,3
3,-79.434403,43.670475,4,148623,2004,D13,Shooting,2004-01-25T05:00:00.000Z,93,Dovercourt-Wallace Emerson-Junction (93),43.670475,-79.434403,4
4,-79.203865,43.823543,5,148619,2004,D42,Shooting,2004-01-25T05:00:00.000Z,131,Rouge (131),43.823543,-79.203865,5


In [6]:
homicide_data.dtypes

NameError: name 'homicide_data' is not defined

In [7]:
#select relevant columns
homicide_data = homicide_rawdata.loc[:, ['Index_','Occurrence_year',"Lat","Long",'Neighbourhood']]
#rename column names 
homicide_data .rename(columns={'Index_':'Homicide_Index',
                          'Occurrence_year':'Homicide_Year',
                          'Lat':'Homicide_Lat',
                          'Long':'Homicide_Lng',
                          'Neighbourhood':'Homicide_Neighbourhood'},inplace=True)
 
#select only 2015 records
homicide_data = homicide_data.loc[homicide_data["Homicide_Year"]==2015]

homicide_data.head()


,Homicide_Index,Homicide_Year,Homicide_Lat,Homicide_Lng,Homicide_Neighbourhood
540,911,2015,43.662422,-79.376717,Cabbagetown-South St.James Town (71)
543,912,2015,43.781338,-79.412674,Newtonbrook East (50)
546,913,2015,43.720859,-79.449371,Yorkdale-Glen Park (31)
632,914,2015,43.652313,-79.406021,Kensington-Chinatown (78)
635,915,2015,43.712559,-79.262207,Clairlea-Birchmount (120)


In [8]:
#check if any missing values and found none
homicide_data.count()

Homicide_Index            59
Homicide_Year             59
Homicide_Lat              59
Homicide_Lng              59
Homicide_Neighbourhood    59
dtype: int64

In [9]:
locations_homicie = homicide_data[["Homicide_Lat", "Homicide_Lng"]]
print(locations_homicie)

     Homicide_Lat  Homicide_Lng
540     43.662422    -79.376717
543     43.781338    -79.412674
546     43.720859    -79.449371
632     43.652313    -79.406021
635     43.712559    -79.262207
638     43.683945    -79.320892
643     43.683945    -79.320892
644     43.701328    -79.548676
645     43.676342    -79.570412
646     43.620514    -79.483269
647     43.732510    -79.586418
648     43.678436    -79.440857
650     43.692280    -79.263382
653     43.695133    -79.303482
656     43.764870    -79.184883
660     43.683121    -79.324768
664     43.665127    -79.454155
668     43.659782    -79.447746
672     43.757725    -79.235268
676     43.757725    -79.235268
680     43.689480    -79.312408
743     43.751030    -79.222259
746     43.708019    -79.341385
782     43.651695    -79.378982
784     43.671673    -79.493752
786     43.641132    -79.423729
789     43.641132    -79.423729
792     43.729393    -79.229729
795     43.668812    -79.447296
798     43.671356    -79.372322
801     

In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations_homicie)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [19]:
# Loop through the Homocide_pd and run a lat/long search for each city
for index, row in homicide_data.iterrows():
    

    latitude = row['Homicide_Lat']
    longitude = row['Homicide_Lng']
#Build the query url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?key=" + gkey + "&latlng=" + f"{latitude},{longitude}"



    # make request
    postalcode_neigh = requests.get(base_url)
    
    
    # convert to json
    postalcode_neigh = postalcode_neigh.json()

    homicide_data.loc[index, "Homicide_Postal Code"] = postalcode_neigh["results"][1]["address_components"][-1]["long_name"]
    homicide_data.loc[index, "Homicide_Neighborhood"] = postalcode_neigh["results"][0]["address_components"][1]["long_name"]

# Visualize to confirm lat lng appear
homicide_data.head()

,Homicide_Index,Homicide_Year,Homicide_Lat,Homicide_Lng,Homicide_Neighbourhood,Postal Code,Neighborhood
540,911,2015,43.662422,-79.376717,Cabbagetown-South St.James Town (71),M4Y 2G6,Old Toronto
543,912,2015,43.781338,-79.412674,Newtonbrook East (50),M2M 4L9,Pemberton Avenue
546,913,2015,43.720859,-79.449371,Yorkdale-Glen Park (31),M6A 2P2,North York
632,914,2015,43.652313,-79.406021,Kensington-Chinatown (78),M5T 1J1,Old Toronto
635,915,2015,43.712559,-79.262207,Clairlea-Birchmount (120),M1K 1E6,Scarborough
